In [ ]:
from numpy import array
from pandas.testing import assert_frame_equal
from sympy import sympify, exp, Symbol, oo, limit, diff

from numpwd.integrate.analytic import SPHERICAL_BASE_SUBS, ANGLE_BASE_SUBS
from numpwd.operators.base import CHANNEL_COLUMNS
from numpwd.operators.expression import decompose_operator
from numpwd.qchannels.spin import get_spin_matrix_element, dict_to_data
from pandas import DataFrame

In [ ]:
spin_expression = sympify("(sigma_ex1 * k1 + sigma_ex2 * k2 + sigma_ex3 * k3)")
spin_expression *= sympify("sigma10 * (sigma21 * k1 + sigma22 * k2 + sigma23 * k3)")
spin_expression

In [ ]:
isospin_expression = sympify("tau10 * tau20")
isospin_expression

## Set up spin momentum expressions

In [ ]:
pi = sympify(["p_i1", "p_i2", "p_i3"])
po = sympify(["p_o1", "p_o2", "p_o3"])
k = sympify(["k_1", "k_2", "k_3"])

Lambda = Symbol("L")
sigma1 = sympify([f"sigma1{n}" for n in range(1, 4)])
sigma2 = sympify([f"sigma2{n}" for n in range(1, 4)])

In [ ]:
pi_m_k = [pp - kk / 2 for pp, kk in zip(pi, k)]
pi_p_k = [pp + kk / 2 for pp, kk in zip(pi, k)]
po_m_k = [pp - kk / 2 for pp, kk in zip(po, k)]
po_p_k = [pp + kk / 2 for pp, kk in zip(po, k)]

In [ ]:
def e1(p1, p2):
    p12 = sum([el ** 2 for el in p1])
    p22 = sum([el ** 2 for el in p2])
    return (p12 - p22) * exp(-(p12 + p22) / Lambda ** 2)


f1 = e1(pi_m_k, po) + e1(pi_p_k, po) + e1(po_m_k, pi) + e1(po_p_k, pi)

In [ ]:
def e2(p1, p2):
    p12 = sum([el ** 2 for el in p1])
    p22 = sum([el ** 2 for el in p2])
    s1_p1 = sum([el1 * el2 for el1, el2 in zip(sigma1, p1)])
    s2_p1 = sum([el1 * el2 for el1, el2 in zip(sigma2, p1)])
    s1_p2 = sum([el1 * el2 for el1, el2 in zip(sigma1, p2)])
    s2_p2 = sum([el1 * el2 for el1, el2 in zip(sigma2, p2)])
    return (s1_p1 * s2_p1 - s1_p2 * s2_p2) * exp(-(p12 + p22) / Lambda ** 2)


f2 = e2(pi_m_k, po) + e2(pi_p_k, po) + e2(po_m_k, pi) + e2(po_p_k, pi)

## Substitutions

In [ ]:
kz_subs = {"k_1": 0, "k_2": 0}
k0_subs = {"k_3": 0, **kz_subs}
substitutions = {**SPHERICAL_BASE_SUBS, **kz_subs}
substitutions

## Operator Definitions

### Operator A

In [ ]:
expr_a = f1 * sympify("sigma10 * sigma20")

$\lim\Lambda \to \infty$

In [ ]:
expr_a_simple = expr_a.subs({Lambda: oo}).simplify()
expr_a_simple

Spherical substitutions and $k_2 = k_1 = 0$ 

In [ ]:
expr_a_subsed = expr_a.subs(substitutions).simplify()
expr_a_subsed

Contributions to different spin channels

In [ ]:
spin_mom_a = DataFrame(get_spin_matrix_element(expr_a_subsed))
spin_mom_a

In [ ]:
spin_mom_a.set_index([col for col in spin_mom_a.columns if col != "expr"]).apply(
    lambda row: row["expr"].subs({Lambda: oo}).expand(), axis=1
).reset_index().rename(columns={0: "expr"})

radius a

In [ ]:
r2a = diff(expr_a, "k_3", "k_3").subs(k0_subs).subs(SPHERICAL_BASE_SUBS).simplify()
r2a

In [ ]:
limit(r2a, Lambda, oo)

### Operator B

In [ ]:
expr_b = sum([el1 * el2 for el1, el2 in zip(sigma1, sigma2)]) * f1

$\lim\Lambda \to \infty$

In [ ]:
expr_b_simple = expr_b.subs({Lambda: oo}).simplify()
expr_b_simple

Spherical substitutions and $k_2 = k_1 = 0$ 

In [ ]:
expr_b_subsed = expr_b.subs(substitutions).simplify()
expr_b_subsed

Contributions to different spin channels

In [ ]:
spin_mom_b = DataFrame(get_spin_matrix_element(expr_b_subsed))
spin_mom_b

In [ ]:
spin_mom_b.set_index([col for col in spin_mom_b.columns if col != "expr"]).apply(
    lambda row: row["expr"].subs({Lambda: oo}).expand(), axis=1
).reset_index().rename(columns={0: "expr"})

radius a

In [ ]:
r2b = diff(expr_b, "k_3", "k_3").subs(k0_subs).subs(SPHERICAL_BASE_SUBS).simplify()
r2b

In [ ]:
limit(r2b, Lambda, oo)

### Operator C

In [ ]:
expr_c = f2

$\lim\Lambda \to \infty$

In [ ]:
expr_c_simple = expr_c.subs({Lambda: oo}).simplify()
expr_c_simple

Spherical substitutions and $k_2 = k_1 = 0$ 

In [ ]:
expr_c_subsed = expr_c.subs(substitutions)  # .simplify()
# expr_c_subsed

Contributions to different spin channels

In [ ]:
spin_mom_c = DataFrame(get_spin_matrix_element(expr_c_subsed, simplify=False))
spin_mom_c

In [ ]:
spin_mom_c.set_index([col for col in spin_mom_c.columns if col != "expr"]).apply(
    lambda row: row["expr"].subs({Lambda: oo}).expand(), axis=1
).reset_index().rename(columns={0: "expr"})

radius a

In [ ]:
r2c = diff(expr_c, "k_3", "k_3").subs(k0_subs).subs(SPHERICAL_BASE_SUBS)#.expand()
#r2c

In [ ]:
r2c.subs({Lambda: oo})